In [ ]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import time
matfile = sio.loadmat('data_mac1_cleaned.mat')
data = matfile['data']

print(data.dtype.names)
print(data.shape)
# format: data['variable name'] [0] [sample index] [0] [time step]
# e.g.: print(data['bus_v']     [0]     [432]      [0]   [124])

use_wandb = True
if use_wandb:
    import wandb, os
    os.environ['WANDB_NOTEBOOK_NAME'] = 'RNN_polar.ipynb'
    wandb.init(project="transient")
    wandb.run.name = 'LSTM_TFinIter'

In [ ]:
# Data pre-processing
np.random.seed(20200412)
n_sample = 1600
data_len = 700
use_mac_ang = True
use_Polar = True
use_cur_sum = True
if use_wandb:
    wandb.config.training_sample = n_sample
    wandb.config.data_len = data_len
    wandb.config.use_mac_ang = use_mac_ang
    wandb.config.use_Polar = use_Polar
    wandb.config.use_cur_sum = use_cur_sum

sample_idx = np.random.permutation(5277)

### load training data ###
if use_cur_sum:
    train_data = np.zeros((data_len, n_sample, 10), dtype=np.float64)
else:
    train_data = np.zeros((data_len, n_sample, 18), dtype=np.float64)
train_label_raw = np.zeros((data_len, n_sample, 8), dtype=np.float64)
n_entry = 0
for i in sample_idx[:n_sample]:
    bus_v = data['bus_v'][0][i].reshape(-1, 1)
    cur = data['cur'][0][i].reshape(-1, 1)
    bus_freq = data['bus_freq'][0][i].reshape(-1, 1)
    mac_ang = data['mac_ang'][0][i].reshape(-1, 1)
    mac_spd = data['mac_spd'][0][i].reshape(-1, 1)
    pelect = data['pelect'][0][i].reshape(-1, 1)
    pmech = data['pmech'][0][i].reshape(-1, 1)
    qelect = data['qelect'][0][i].reshape(-1, 1)
    length = qelect.shape[0]

    if not use_mac_ang:
        mac_ang = np.zeros_like(mac_ang)

#     if use_cur_sum:
#         cur = np.sum(cur, axis=1).reshape(-1, 1)

    bus_v_ang = np.unwrap(np.angle(bus_v).reshape(-1)).reshape(-1,1)
    cur_ang = np.unwrap(np.angle(cur).reshape(-1)).reshape(-1,1)
    tmp_train_data = np.hstack([np.abs(bus_v), bus_v_ang, np.abs(cur), cur_ang, bus_freq, mac_ang, mac_spd, pelect, pmech, qelect])
    tmp_train_data = tmp_train_data[:data_len+1, :]
    
    tmp_train_label = np.delete(tmp_train_data, 0, 0)  # delete the first sample(shift the curve left)

    # delete the currents(we don't need to predict that)
    if use_cur_sum:
        tmp_train_label = np.delete(tmp_train_label, np.arange(2, 4), 1)
    else:
        tmp_train_label = np.delete(tmp_train_label, np.arange(2, 12), 1)
    tmp_train_data = np.delete(tmp_train_data, -1, 0)  # delete the last sample because there's no corresponding label

    length = tmp_train_data.shape[0]
    train_data[:, n_entry, :] = tmp_train_data
    train_label_raw[:, n_entry, :] = tmp_train_label
    n_entry += 1
del tmp_train_data
del tmp_train_label

### load testing data ###
if use_cur_sum:
    test_data = np.zeros((data_len, 400, 10), dtype=np.float64)
else:
    test_data = np.zeros((data_len, 400, 18), dtype=np.float64)
test_label_raw = np.zeros((data_len, 400, 8), dtype=np.float64)

n_entry = 0
for i in sample_idx[4800:5200]:
    bus_v = data['bus_v'][0][i].reshape(-1, 1)
    cur = data['cur'][0][i].reshape(-1, 1)
    bus_freq = data['bus_freq'][0][i].reshape(-1, 1)
    mac_ang = data['mac_ang'][0][i].reshape(-1, 1)
    mac_spd = data['mac_spd'][0][i].reshape(-1, 1)
    pelect = data['pelect'][0][i].reshape(-1, 1)
    pmech = data['pmech'][0][i].reshape(-1, 1)
    qelect = data['qelect'][0][i].reshape(-1, 1)
    length = qelect.shape[0]

    if not use_mac_ang:
        mac_ang = np.zeros_like(mac_ang)

#     if use_cur_sum:
#         cur = np.sum(cur, axis=1).reshape(-1, 1)

    bus_v_ang = np.unwrap(np.angle(bus_v).reshape(-1)).reshape(-1,1)
    cur_ang = np.unwrap(np.angle(cur).reshape(-1)).reshape(-1,1)
    tmp_test_data = np.hstack([np.abs(bus_v), bus_v_ang, np.abs(cur), cur_ang, bus_freq, mac_ang, mac_spd, pelect, pmech, qelect])
    tmp_test_data = tmp_test_data[:data_len+1, :]
    
    tmp_test_label = np.delete(tmp_test_data, 0, 0)  # delete the first sample(shift the curve left)
    if use_cur_sum:
        tmp_test_label = np.delete(tmp_test_label, np.arange(2, 4), 1)
    else:
        tmp_test_label = np.delete(tmp_test_label, np.arange(2, 12), 1)
    tmp_test_data = np.delete(tmp_test_data, -1, 0)  # delete the last sample because there's no corresponding label
    length = tmp_test_data.shape[0]
    test_data[:, n_entry, :] = tmp_test_data
    test_label_raw[:, n_entry, :] = tmp_test_label

    n_entry += 1
del tmp_test_data
del tmp_test_label

# train_label_raw[:, :2] = train_data[:, :2]
# test_label_raw[:, :2] = test_data[:, :2]

# compute difference
if use_cur_sum:
    train_label = train_label_raw# - train_data[:, [0, 1, 4, 5, 6, 7, 8, 9]]
    test_label = test_label_raw# - test_data[:, [0, 1, 4, 5, 6, 7, 8, 9]]
else:
    train_label = train_label_raw# - train_data[:, [0, 1, 12, 13, 14, 15, 16, 17]]
    test_label = test_label_raw# - test_data[:, [0, 1, 12, 13, 14, 15, 16, 17]]

# clip angle to [-pi, pi]
# train_label[:, 1] = np.mod(train_label[:, 1]+np.pi, 2*np.pi) - np.pi
# test_label[:, 1] = np.mod(test_label[:, 1]+np.pi, 2*np.pi) - np.pi

# train_label[:, :3] = 0
# test_label[:, :3] = 0

# # normalize testing data
label_mean = np.mean(train_label.reshape(n_sample*data_len,-1), axis=0)
label_std = np.std(train_label.reshape(n_sample*data_len,-1), axis=0)
label_std[np.less(label_std, 1e-7)] = 1e-7
# label_mean[1] = 0  # remove normalization on angle of bus_v
# label_std[1] = 1
train_label = (train_label - label_mean) / label_std
test_label = (test_label - label_mean) / label_std

# # normalize training data
data_mean = np.mean(train_data.reshape(n_sample*data_len,-1), axis=0)
data_std = np.std(train_data.reshape(n_sample*data_len,-1), axis=0)
data_std[np.less(data_std, 1e-7)] = 1e-7
train_data = (train_data - data_mean) / data_std
test_data = (test_data - data_mean) / data_std

print('Train data set size:', train_data.shape)
print('Test data set size:', test_data.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class RNNNetwork(nn.Module):

    def __init__(self):
        super(RNNNetwork, self).__init__()
        self.hidden_dim = 256
        self.n_layers = 2
        
        if use_wandb:
            wandb.config.architecture = 'LSTM'
            wandb.config.n_layers = self.n_layers
            wandb.config.hidden_dim = self.hidden_dim

#         self.initial_hidden = torch.zeros(self.n_layers, 1, self.hidden_dim).to(device)
#         self.initial_cell = torch.zeros(self.n_layers, 1, self.hidden_dim).to(device)
#         self.initial_state = (self.initial_hidden, self.initial_cell)

#         self.initial_cell = torch.zeros(self.n_layers, 1, self.hidden_dim).to(device)

        self.lstm = nn.LSTM(input_size=10, hidden_size=self.hidden_dim, num_layers=self.n_layers)
#         self.gru = nn.GRU(input_size=10, hidden_size=self.hidden_dim, num_layers=self.n_layers)

        self.outputlayer = nn.Linear(self.hidden_dim, 8)

    def forward(self, x, states=None):
        data_len = x.size()[0]
        data_batch = x.size()[1]

        # LSTM
        (hidden_state, cell_state) = (None, None) if states is None else states
        if hidden_state is None:
            hidden_state = torch.zeros(self.n_layers, data_batch, self.hidden_dim).to(device)
        if cell_state is None:
            cell_state = torch.zeros(self.n_layers, data_batch, self.hidden_dim).to(device)

        rnn_out, (hidden_state, cell_state) = self.lstm(x, (hidden_state, cell_state))
        states = (hidden_state, cell_state)

        # GRU
#         if states is None:
#             states = torch.zeros(self.n_layers, data_batch, self.hidden_dim).to(device)
#         rnn_out, states = self.gru(x, states)

        pred = self.outputlayer(rnn_out.view(-1, self.hidden_dim))
        return pred.view(data_len, data_batch, 8), states

    def initial_states(self):
        return None


def weights_init(m):
    if isinstance(m, nn.Linear):
        #         torch.nn.init.kaiming_normal_(m.weight)
        #         torch.nn.init.normal_(m.weight, mean=0, std=0.001)
        torch.nn.init.normal_(m.bias, mean=0, std=0.001)
        nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
#         nn.init.kaiming_uniform_(m.bias, mode='fan_in', nonlinearity='relu')

    if isinstance(m, nn.BatchNorm1d):
        torch.nn.init.zeros_(m.bias)
        torch.nn.init.ones_(m.weight)


torch.manual_seed(20200410)
if torch.cuda.is_available():
    print('GPU is available. Will be applied.')
    use_GPU = True
    device = torch.device('cuda')

    torch.cuda.empty_cache()
    torch.cuda.manual_seed(20200410)
    torch.cuda.manual_seed_all(20200410)
    torch.backends.cudnn.deterministic = True
else:
    use_GPU = False
    device = torch.device('cpu')

model = RNNNetwork().float()
model.apply(weights_init)
# model.load_state_dict(torch.load('GRU_CL0.75.pt'))
model = model.to(device)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')


batchsize = 200
criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01*2, momentum=0.9, nesterov=True)
optimizer = optim.Adam(model.parameters())
# optimizer = optim.RMSprop(model.parameters(), lr=0.0001)

train_data_torch = torch.from_numpy(train_data).float()
train_label_torch = torch.from_numpy(train_label).float()
test_data_torch = torch.from_numpy(test_data).float()
test_label_torch = torch.from_numpy(test_label).float()


def test(data_torch, label_torch):
    loss = None
    prediction = None
    model.eval()
    test_batchsize = 400
    with torch.no_grad():
        for j in range(0, data_torch.shape[1], test_batchsize):
            inputs_torch = data_torch[:, j:j+test_batchsize, :]
            labels_torch = label_torch[:, j:j+test_batchsize, :]

            inputs_torch = inputs_torch.to(device)
            labels_torch = labels_torch.to(device)

            # forward
            pred_torch, _ = model(inputs_torch)
            loss_torch = criterion(pred_torch, labels_torch)

            if use_GPU:
                pred_torch = pred_torch.cpu()
            pred = pred_torch.data.numpy()

            loss = np.array([loss_torch.item()]) if loss is None else np.hstack([loss, loss_torch.item()])
            prediction = pred if prediction is None else np.concatenate([prediction, pred], axis=1)
    return loss.mean(), prediction

def evaluation(eval_data_torch, eval_label_raw):
    t_max = 699
    eval_batchsize = 400
    rse = np.zeros((400,8))

    model.eval()
    with torch.no_grad():
        for i in range(0,400,eval_batchsize):
            rnn_states = model.initial_states()
            output_data = np.zeros((t_max, eval_batchsize, 8))
            input_data_torch = eval_data_torch[0, i:i+eval_batchsize, :].reshape(1,eval_batchsize,-1).to(device)

            for t in range(0, t_max):
                output_data_torch, rnn_states = model(input_data_torch, rnn_states)
                output_data[t,:] = output_data_torch.data.cpu().numpy()
                input_data_torch = torch.cat((output_data_torch[0,:,0:2], eval_data_torch[t+1,i:i+eval_batchsize,2:4].to(device), output_data_torch[0,:,2:]), dim=1).reshape(1,eval_batchsize,-1)

            output_data = output_data * label_std + label_mean

            for b in range(eval_batchsize):
                for q in range(8):
                    if q == 6:
                        continue
                    truth_curve = eval_label_raw[:t_max, i+b, q]
                    pred_curve = output_data[:t_max, b, q]
                    rse[i+b,q] = np.linalg.norm(truth_curve - pred_curve, 2)/ (np.linalg.norm(truth_curve-truth_curve.mean(), 2) + 1e-6)
    return rse

loss, _ = test(train_data_torch, train_label_torch)
loss_test, _ = test(test_data_torch, test_label_torch)
print('Before training: train Loss = %.3e, test Loss = %.3e' % (loss, loss_test))

trained_epochs = 1
loss_curve = []

In [ ]:
%matplotlib inline
teacher_forcing_ratio = 0.75
train_for_epochs = 1000
if use_wandb:
    wandb.config.teacher_forcing = teacher_forcing_ratio
    wandb.config.teacher_forcing_ratio_decay = False

start_time = time.time()

for i in range(trained_epochs, trained_epochs+train_for_epochs):
    model.train()
#     teacher_forcing_ratio *= 0.9975 #  0.9966

    for j in range(0, n_sample, batchsize):
#         input_batch = train_data_torch[:data_len, j:j+batchsize, :].reshape(data_len,batchsize,-1).to(device)
#         outputs = torch.zeros((data_len,batchsize,8), dtype=torch.float32, device=device)
        rnn_states = model.initial_states()

        loss = 0

        use_teacher_forcing = np.random.random()<teacher_forcing_ratio
        if use_teacher_forcing:
#             with total teacher forcing, 20x faster
            inputs = train_data_torch[:data_len, j:j+batchsize, :].reshape(data_len,batchsize,-1).to(device)
            outputs, rnn_states = model(inputs, rnn_states)
            labels = train_label_torch[:, j:j+batchsize, :].to(device)
            loss = criterion(outputs, labels)
        else:
            for t in range(data_len):
                inputs = train_data_torch[t, j:j+batchsize, :].reshape(1,batchsize,-1).to(device)
                if t>0:
                    # tracher forcing: replacing inputs with ground truth
                    inputs[:,:,[0,1,4,5,6,7,8,9]] = outputs[0,:,:].detach()

                # forward
                outputs, rnn_states = model(inputs, rnn_states)

                labels = train_label_torch[t, j:j+batchsize, :].reshape(1,batchsize,-1).to(device)
                loss += criterion(outputs, labels)
            loss /= data_len
#         print(use_teacher_forcing, loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient clip
        torch.nn.utils.clip_grad_value_(model.parameters(), 0.5)

        optimizer.step()

    if i % 10 == 0:
        loss, _ = test(train_data_torch, train_label_torch)
        loss_test, _ = test(test_data_torch, test_label_torch)
        rse = evaluation(test_data_torch, test_label_raw)

        loss_curve.append([i, loss, loss_test])
        if use_wandb:
            wandb.log({"Epoch": i, "Train Loss": loss, "Test Loss": loss_test, 'Eval RSE':np.median(rse,axis=0).mean()})


    if i % 10 == 0:
        ETA = (time.time() - start_time) / (i - trained_epochs + 1) * (trained_epochs+train_for_epochs-i) / 60
        print(time.strftime('%H:%M:%S  ', time.localtime(time.time())), end="")
        print('ETA:{:.1f}mins '.format(ETA), end="")
        print('Epoch {}: train Loss = {:.3e}, test Loss = {:.3e}'.format(i, loss, loss_test))


trained_epochs = trained_epochs+train_for_epochs
print('Training process took {:.1f} mins.'.format((time.time()-start_time)/60))

if len(loss_curve) > 0:
    plt.figure(0, figsize=(15, 5))
    plt.plot(np.array(loss_curve)[:, 0], np.array(loss_curve)[:, 1], 'r', label="Train")
    plt.plot(np.array(loss_curve)[:, 0], np.array(loss_curve)[:, 2], 'b', label="Test")
    plt.yscale('log')
    plt.grid(True)
    plt.legend()
#     plt.show()

print('GPU memory stats: Max Cached {:.0f} Mb, Max Reserved {:.0f} Mb, Max Allocated {:.0f} Mb'.format(
    torch.cuda.max_memory_cached(0)/1024/1024, torch.cuda.max_memory_reserved(0)/1024/1024, torch.cuda.max_memory_allocated(0)/1024/1024))

_, prediction_train_raw = test(train_data_torch, train_label_torch)
_, prediction_test_raw = test(test_data_torch, test_label_torch)

if use_cur_sum:
    prediction_train = prediction_train_raw * label_std + label_mean
    prediction_test = prediction_test_raw * label_std + label_mean
else:
    prediction_train = prediction_train_raw * label_std + label_mean
    prediction_test = prediction_test_raw * label_std + label_mean
    
mse_train = np.mean(np.mean(np.square(prediction_train - train_label_raw), axis=1), axis=0)
mse_test = np.mean(np.mean(np.square(prediction_test - test_label_raw), axis=1), axis=0)

print('Denormalized: train Loss = %.3e, test Loss = %.3e' % (mse_train.mean(), mse_test.mean()))

se_train = np.linalg.norm(prediction_train - train_label_raw, 2, axis=0) / (np.linalg.norm(train_label_raw-train_label_raw.mean(axis=0), 2, axis=0) + 1e-5)
se_test = np.linalg.norm(prediction_test - test_label_raw, 2, axis=0) / (np.linalg.norm(test_label_raw-test_label_raw.mean(axis=0), 2, axis=0) + 1e-5)
rse_train = np.median(se_train, axis=0)
rse_test = np.median(se_test, axis=0)
print('Relative SE after de-normalization: train {:.3e}, test {:.3e}'.format(rse_train.mean(), rse_test.mean()))

### Evaluation
rse = evaluation(test_data_torch, test_label_raw)
print(np.median(rse,axis=0))
print('Overall:', np.median(rse,axis=0).mean())
    
if use_wandb:
    wandb.log({"Training Time": (time.time()-start_time)/60})
    wandb.log({"Train Relative SE": rse_train.mean()})
    wandb.log({"Test Relative SE": rse_test.mean()})
    wandb.log({"Detailed Relative SE": rse_train})
    wandb.log({"Detailed Relative SE": rse_test})
    
    wandb.log({"Prediction RSE": np.median(rse,axis=0)})
    wandb.log({"Overall": np.median(rse,axis=0).mean()})

In [ ]:
# torch.save(model.state_dict(), 'GRU_CL0.75.pt')

# Save model to wandb
if use_wandb:
    import os
    torch.save(model.state_dict(), os.path.join(wandb.run.dir, 'model.pt'))

In [ ]:
c = ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'orange', 'b', 'b', 'b']
c2 = ['C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'darkorange', 'C0', 'C0', 'C0']
x = np.arange(8)
# xticks=('0\nR:bus_v', '1\nI:bus_v', '2\nR:cur1', '3\nR:cur2', '4\nR:cur3', '5\nR:cur4', '6\nR:cur5', '7\nI:cur1', '8\nI:cur2', '9\nI:cur3', \
#                        '10\nI:cur4', '11\nI:cur5', '12\nbus_freq', '13\nmac_ang', '14\nmac_spd', '15\npelect', '16\npmech', '17\nqelect')
xticks = ('0\nR:bus_v', '1\nI:bus_v', '2\nbus_freq', '3\nmac_ang', '4\nmac_spd', '5\npelect', '6\npmech', '7\nqelect')
width = 0.35

# plt.figure(1,figsize=(15,7))
# plt.bar(range(8), data_std, color=c)
# plt.xticks(range(8), ('0\nR:bus_v', '1\nI:bus_v', '2\nR:cur1', '3\nR:cur2', '4\nR:cur3', '5\nR:cur4', '6\nR:cur5', '7\nI:cur1', '8\nI:cur2', '9\nI:cur3', \
#                        '10\nI:cur4', '11\nI:cur5', '12\nbus_freq', '13\nmac_ang', '14\nmac_spd', '15\npelect', '16\npmech', '17\nqelect'))
# plt.grid(True)
# plt.xlim([-0.5,17.5])
# # plt.ylim([0,0.001])
# plt.title('Standard Deviations on different components')

mse_train_n = np.mean(np.mean(np.square(prediction_train_raw - train_label), axis=1), axis=0)
mse_test_n = np.mean(np.mean(np.square(prediction_test_raw - test_label), axis=1), axis=0)

plt.figure(2, figsize=(15, 7))
plt.bar(x-width/2, mse_train_n, width, label='Train', color=c)
plt.bar(x+width/2, mse_test_n, width, label='Test', color=c2)
plt.xticks(x, xticks)
plt.grid(True)
plt.xlim([-0.5, 7.5])
plt.legend()
# plt.ylim([0,1e-6])
plt.title('MSE on different components, normalized')


plt.figure(3, figsize=(15, 7))
plt.bar(x-width/2, rse_train, width, label='Train', color=c)
plt.bar(x+width/2, rse_test, width, label='Test', color=c2)
plt.xticks(x, xticks)
plt.grid(True)
plt.xlim([-0.5, 7.5])
plt.legend()
plt.yscale('log')
# plt.ylim([0,1e-1])
plt.title('Relative SE on different components')
print('{:.2e},{:.2e},{:.2e},{:.2e},{:.2e},{:.2e},{:.2e},{:.2e}'.format(rse_train[0], rse_train[1], rse_train[2], rse_train[3], rse_train[4], rse_train[5], rse_train[6], rse_train[7]))
print('{:.2e},{:.2e},{:.2e},{:.2e},{:.2e},{:.2e},{:.2e},{:.2e}'.format(rse_test[0], rse_test[1], rse_test[2], rse_test[3], rse_test[4], rse_test[5], rse_test[6], rse_test[7]))

# plt.show()

In [ ]:
replace_mode = 0 # 0:no cheating  1: replace accord to the list  2: replace all others  3: replace all, including the plotting one
# entry = np.floor(np.random.uniform(low=0, high=1600, size=(1, 1))).astype(int).reshape(-1)
entry = np.floor(np.random.uniform(low=4800, high=5200, size=(1, 1))).astype(int).reshape(-1)
# entry=[5142]
# entry =[2108]  % contigency at Line 5
# entry=[4880]  % another contigency at Line 5
entry = sample_idx[entry]
t_max = 700
if use_cur_sum:
    quantities = [0,1,4,5,6,7,8,9]  # in data indices
    quantities_transform = [0, 1, -1, -1, 2, 3, 4, 5, 6, 7]  # transform data indices to label indices
    quantites_names = ['Mag:bus_v', 'Ang:bus_v', 'M:cur_sum', 'A:cur_sum', 'bus_freq', 'mac_ang', 'mac_spd', 'pelect', 'pmech', 'qelect']
    quantities_replace = [0,1]  # replace these in data indices
#     quantities_replace = [0,1,4,5,6,7,8,9]  # replace these in data indices
else:
    quantities = [0,1,12,13,14,15,16,17]
    quantities_transform = [0, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7]
    quantites_names = ['Mag:bus_v', 'Ang:bus_v', 'M:cur1', 'M:cur2', 'M:cur3', 'M:cur4', 'M:cur5', 'A:cur1', 'A:cur2', 'A:cur3', 'A:cur4', 'A:cur5', 'bus_freq', 'mac_ang', 'mac_spd', 'pelect', 'pmech', 'qelect']
    quantities_replace = [0,1,12,13,15,16,17]
    
if replace_mode == 0:
    print('No Cheating.')
elif replace_mode == 1:
    print('Replacing quantites:')
    for q in quantities_replace:
        print('    [{:d}]'.format(q),quantites_names[q])
    print('Keeping quantites:')
    for q in quantities:
        if not (q in quantities_replace):
            print('    [{:d}]'.format(q),quantites_names[q])
    print()
elif replace_mode == 2:
       print('Replacing all other quantities except the plotting one.')
elif replace_mode == 3:
       print('Replacing all quantities.')

model.eval()
for i in entry:    
    bus_v = data['bus_v'][0][i].reshape(-1, 1)
    cur = data['cur'][0][i].reshape(-1, 1)
    bus_freq = data['bus_freq'][0][i].reshape(-1, 1)
    mac_ang = data['mac_ang'][0][i].reshape(-1, 1)
    mac_spd = data['mac_spd'][0][i].reshape(-1, 1)
    pelect = data['pelect'][0][i].reshape(-1, 1)
    pmech = data['pmech'][0][i].reshape(-1, 1)
    qelect = data['qelect'][0][i].reshape(-1, 1)

    if not use_mac_ang:
        mac_ang = np.zeros_like(mac_ang)

#     if use_cur_sum:
#         cur = np.sum(cur, axis=1).reshape(-1, 1)
        
    bus_v_ang = np.unwrap(np.angle(bus_v).reshape(-1)).reshape(-1,1)
    cur_ang = np.unwrap(np.angle(cur).reshape(-1)).reshape(-1,1)
    tmp_test_data = np.hstack([np.abs(bus_v), bus_v_ang, np.abs(cur), cur_ang, bus_freq, mac_ang, mac_spd, pelect, pmech, qelect])
#     tmp_test_data = np.array(tmp_test_data, dtype=np.float32)
    
    tmp_test_label = np.delete(tmp_test_data, 0, 0)  # delete the first sample(shift the curve left)
    if use_cur_sum:
        tmp_test_label = np.delete(tmp_test_label, np.arange(2, 4), 1)
    else:
        tmp_test_label = np.delete(tmp_test_label, np.arange(2, 12), 1)
    tmp_test_data = np.delete(tmp_test_data, -1, 0)
    
    
    tmp_test_data = (tmp_test_data - data_mean) / data_std
    tmp_test_data = torch.from_numpy(tmp_test_data).to(device).float()

    for q in range(10):
        if quantites_names[q] == 'pmech':
            continue
        if not (q in quantities):
            plt.figure(figsize=(12, 8))
            if q==2:
                plt.plot(range(t_max-1), np.abs(cur)[1:t_max], 'r')
            if q==3:
                plt.plot(range(t_max-1), cur_ang[1:t_max], 'r')
            plt.grid(True)
            plt.title('#{:d}: [{:s}]'.format(i, quantites_names[q]))
            continue
#         if q in quantities_replace:
#             continue
        rnn_states = model.initial_states()
        output_data = np.zeros_like(tmp_test_label)
        input_data_torch = tmp_test_data[0, :].reshape(1,1,-1)
        
        for t in range(0, t_max):
            output_data_torch, rnn_states = model(input_data_torch, rnn_states)
            output_data[t,:] = output_data_torch.data.cpu().numpy()
            
            input_data_torch = torch.cat((output_data_torch[0,0,0:2], tmp_test_data[t+1,2:4], output_data_torch[0,0,2:])).reshape(1,1,-1)

            for r in quantities:
                if ((replace_mode == 1) and (r in quantities_replace)) or ((replace_mode == 2) and (r != q)) or (replace_mode == 3):
                    input_data_torch[0,0,r] = tmp_test_data[t+1, r]

        output_data = output_data * label_std + label_mean

        truth_curve = tmp_test_label[:t_max, quantities_transform[q]]
        pred_curve = output_data[:t_max, quantities_transform[q]]
        plt.figure(figsize=(12, 8))
        plt.plot(range(t_max), truth_curve, 'r', label='Truth')
        plt.plot(range(t_max), pred_curve, 'b', label='Pred')
        plt.legend()
        plt.grid(True)

        rmse = np.sqrt(np.mean(np.square(truth_curve - pred_curve)))
        rse = np.linalg.norm(truth_curve - pred_curve, 2)/ np.linalg.norm(truth_curve-truth_curve.mean(), 2)
        plt.title('#{:d}: [{:s}] Pred RMSE={:.4e}, Relative SE = {:.4e}'.format(i, quantites_names[q], rmse, rse))
        
# plt.show()

# sio.savemat('example1.mat',{'label':tmp_test_label, 'data':output_data})

In [ ]:
print(adbe)

In [ ]:
entry = np.floor(np.random.uniform(low=0, high=1500, size=(1, 1))).astype(int).reshape(-1)
# entry=[3129]
# entry =[4108]
# entry = [5]
entry = sample_idx[entry]
t_max = 700
if use_cur_sum:
    quantities = [0,1,4,5,6,7,8,9]
    quantities_transform = [0, 1, -1, -1, 2, 3, 4, 5, 6, 7]
    quantites_names = ['Mag:bus_v', 'Ang:bus_v', 'M:cur_sum', 'A:cur_sum', 'bus_freq', 'mac_ang', 'mac_spd', 'pelect', 'pmech', 'qelect']
else:
    quantities = [0,1,12,13,14,15,16,17]
    quantities_transform = [0, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7]
    quantites_names = ['Mag:bus_v', 'Ang:bus_v', 'M:cur1', 'M:cur2', 'M:cur3', 'M:cur4', 'M:cur5', 'A:cur1', 'A:cur2', 'A:cur3', 'A:cur4', 'A:cur5', 'bus_freq', 'mac_ang', 'mac_spd', 'pelect', 'pmech', 'qelect']

model_cpu.eval()
for i in entry:    
    bus_v = data['bus_v'][0][i].reshape(-1, 1)
    cur = data['cur'][0][i].reshape(5, -1).T
    bus_freq = data['bus_freq'][0][i].reshape(-1, 1)
    mac_ang = data['mac_ang'][0][i].reshape(-1, 1)
    mac_spd = data['mac_spd'][0][i].reshape(-1, 1)
    pelect = data['pelect'][0][i].reshape(-1, 1)
    pmech = data['pmech'][0][i].reshape(-1, 1)
    qelect = data['qelect'][0][i].reshape(-1, 1)

    if not use_mac_ang:
        mac_ang = np.zeros_like(mac_ang)

    if use_cur_sum:
        cur = np.sum(cur, axis=1).reshape(-1, 1)

    tmp_test_data = np.hstack([np.abs(bus_v), np.angle(bus_v), np.abs(cur), np.angle(cur), bus_freq, mac_ang, mac_spd, pelect, pmech, qelect])
    tmp_test_data = np.delete(tmp_test_data, -1, 0)
    
    q=6
    for start_t in range(0,600,100):
        prediction2 = np.zeros((t_max, 8))
        input_data_raw = tmp_test_data[[start_t], :]
        for t in range(start_t+1, t_max):
            input_data = (input_data_raw - data_mean) / data_std
            input_data_torch = torch.from_numpy(input_data).float()
            output_data_torch = model_cpu(input_data_torch)
            if use_cur_sum:
                prediction2[t] = output_data_torch.data.numpy() * label_std + label_mean + input_data_raw[:, [0, 1, 4, 5, 6, 7, 8, 9]]
                input_data_raw = np.concatenate([prediction2[t, [0, 1]], tmp_test_data[t, [2, 3]], prediction2[t, [2, 3, 4, 5, 6, 7]]])
            else:
                prediction2[t] = output_data_torch.data.numpy() * label_std + label_mean + input_data_raw[:, [0, 1, 12, 13, 14, 15, 16, 17]]
                input_data_raw = np.concatenate([prediction2[t, [0, 1]], tmp_test_data[t, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], prediction2[t, [2, 3, 4, 5, 6, 7]]])

            for r in quantities:
                if r != q:
                    input_data_raw[r] = tmp_test_data[t, r]

            input_data_raw[1] = np.mod(input_data_raw[1]+np.pi, 2*np.pi) - np.pi
            input_data_raw = input_data_raw.reshape(1, -1)
#         print(prediction2.shape)
        
        plt.figure(1, figsize=(24, 16))
        if (start_t==0):
            plt.plot(tmp_test_data[:t_max,q], 'r', label='Truth')
            plt.title('#{:d}: [{:s}] Pred RMSE={:.4e}'.format(i, quantites_names[q], np.sqrt(np.mean(np.square(prediction2[1:, quantities_transform[q]] - tmp_test_data[1:t_max,q].reshape(-1)))) ))

        plt.plot(np.arange(start_t+1,t_max), prediction2[start_t+1:, quantities_transform[q]], label='Pred_'+str(start_t))
#         plt.plot([0, t_max], [tmp_test_data[0,q], tmp_test_data[0,q]], 'g-.', label='Benchmark')
        plt.legend()
        plt.grid(True)
    #     plt.ylim([0.97, 1.03])
        #         plt.title('#{:d}: [{:s}] Pred MSE={:.4e} Benchmark MSE={:.4e}'.format(i, quantites_names[q], np.mean(np.square(prediction2[1:, quantities_transform[q]] - tmp_test_data[1:t_max,q].reshape(-1))), np.mean(np.square(tmp_test_data[0,q] - tmp_test_data[1:t_max,q].reshape(-1))) ))
    
# plt.show()

In [ ]:
# entry = np.floor(np.random.uniform(low=0, high=5864, size=(1, 1))).astype(int).reshape(-1)
# entry=[3129]
entry =[4951]
# entry = [sample_idx[1]]
t_max = 700
quantities_replace = []
if use_cur_sum:
    quantities = [0,1,4,5,6,7,8,9]
    quantities_transform = [0, 1, -1, -1, 2, 3, 4, 5, 6, 7]
    quantites_names = ['Mag:bus_v', 'Ang:bus_v', 'M:cur_sum', 'A:cur_sum', 'bus_freq', 'mac_ang', 'mac_spd', 'pelect', 'pmech', 'qelect']
#     quantities_replace = [0,1,4,5,7,8,9]
else:
    quantities = [0,1,12,13,14,15,16,17]
    quantities_transform = [0, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7]
    quantites_names = ['Mag:bus_v', 'Ang:bus_v', 'M:cur1', 'M:cur2', 'M:cur3', 'M:cur4', 'M:cur5', 'A:cur1', 'A:cur2', 'A:cur3', 'A:cur4', 'A:cur5', 'bus_freq', 'mac_ang', 'mac_spd', 'pelect', 'pmech', 'qelect']
    quantities_replace = [0,1,12,13,15,16,17]


print('Replacing quantites:')
for q in quantities_replace:
    print('    [{:d}]'.format(q),quantites_names[q])
print('Keeping quantites:')
for q in quantities:
    if not (q in quantities_replace):
        print('    [{:d}]'.format(q),quantites_names[q])
print()


model_cpu = model.cpu()
model_cpu.eval()
for i in entry:    
    bus_v = data['bus_v'][0][i].reshape(-1, 1)
    cur = data['cur'][0][i].reshape(5, -1).T
    bus_freq = data['bus_freq'][0][i].reshape(-1, 1)
    mac_ang = data['mac_ang'][0][i].reshape(-1, 1)
    mac_spd = data['mac_spd'][0][i].reshape(-1, 1)
    pelect = data['pelect'][0][i].reshape(-1, 1)
    pmech = data['pmech'][0][i].reshape(-1, 1)
    qelect = data['qelect'][0][i].reshape(-1, 1)

    if not use_mac_ang:
        mac_ang = np.zeros_like(mac_ang)

    if use_cur_sum:
        cur = np.sum(cur, axis=1).reshape(-1, 1)

    tmp_test_data = np.hstack([np.abs(bus_v), np.angle(bus_v), np.abs(cur), np.angle(cur), bus_freq, mac_ang, mac_spd, pelect, pmech, qelect])
    tmp_test_data = np.delete(tmp_test_data, -1, 0)
    

    prediction2 = np.zeros((t_max, 8))
    input_data_raw = tmp_test_data[[0], :]
    for t in range(1, t_max):
        input_data = (input_data_raw - data_mean) / data_std
        input_data_torch = torch.from_numpy(input_data).float()
        output_data_torch = model_cpu(input_data_torch)
        if use_cur_sum:
            prediction2[t] = output_data_torch.data.numpy() * label_std + label_mean + input_data_raw[:, [0, 1, 4, 5, 6, 7, 8, 9]]
            input_data_raw = np.concatenate([prediction2[t, [0, 1]], tmp_test_data[t, [2, 3]], prediction2[t, [2, 3, 4, 5, 6, 7]]])
        else:
            prediction2[t] = output_data_torch.data.numpy() * label_std + label_mean + input_data_raw[:, [0, 1, 12, 13, 14, 15, 16, 17]]
            input_data_raw = np.concatenate([prediction2[t, [0, 1]], tmp_test_data[t, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], prediction2[t, [2, 3, 4, 5, 6, 7]]])

        input_data_raw[quantities_replace] = tmp_test_data[t, quantities_replace]

        input_data_raw[1] = np.mod(input_data_raw[1]+np.pi, 2*np.pi) - np.pi
        input_data_raw = input_data_raw.reshape(1, -1)
#         print(prediction2.shape)

    for q in quantities:
        if q==14 or not (q in quantities_replace):
            plt.figure(figsize=(12, 8))
            plt.plot(tmp_test_data[:t_max,q], 'r', label='Truth')
            plt.plot(np.arange(1,t_max), prediction2[1:, quantities_transform[q]], 'b', label='Pred')
            plt.legend()
            plt.grid(True)
        #     plt.ylim([0.97, 1.03])
            plt.title('#{:d}: [{:s}] Pred RMSE={:.4e}'.format(i, quantites_names[q], np.sqrt(np.mean(np.square(prediction2[1:, quantities_transform[q]] - tmp_test_data[1:t_max,q].reshape(-1)))) ))
    
# plt.show()

# for i in entry:
#     print('#{:d} sample is '.format(i), end='')
#     if entry[0] in sample_idx[:n_sample]:
#         print('in training set.')
#     else:
#         print('not in training set.')
    
#     bus_v = data['bus_v'][0][i].reshape(-1, 1)
#     cur = data['cur'][0][i].reshape(5, -1).T
#     bus_freq = data['bus_freq'][0][i].reshape(-1, 1)
#     mac_ang = data['mac_ang'][0][i].reshape(-1, 1)
#     mac_spd = data['mac_spd'][0][i].reshape(-1, 1)
#     pelect = data['pelect'][0][i].reshape(-1, 1)
#     pmech = data['pmech'][0][i].reshape(-1, 1)
#     qelect = data['qelect'][0][i].reshape(-1, 1)
    
# #     mac_ang = np.zeros_like(mac_ang)

#     tmp_test_data = np.hstack([np.abs(bus_v), np.angle(bus_v), np.abs(cur), np.angle(cur), bus_freq, mac_ang, mac_spd, pelect, pmech, qelect])
# #     tmp_test_data = (tmp_test_data - data_mean) / data_std

# #     tmp_test_label = np.delete(tmp_test_data, 0, 0)
#     tmp_test_data = np.delete(tmp_test_data, -1, 0)

# #     tmp_test_data[:,13] = 0

# #     input_data_torch = torch.from_numpy(tmp_test_data).float()
# #     prediction = model_cpu(input_data_torch).data.numpy()
# #     prediction = prediction * label_std + label_mean + tmp_test_data * data_std + data_mean
# #     print(prediction.shape)
# #     mac_spd = mac_spd[1:]

# #     plt.figure(figsize = (12,8))
# #     plt.plot(mac_spd,'r')
# #     plt.plot(prediction[:,14], 'b')
# #     plt.legend(['Truth','Pred'])
# #     plt.grid(True)
# #     plt.title('#%d: MSE=%.4e benchmark=%.4e'%(i, np.mean(np.square(prediction[:,14] - mac_spd)), np.mean(np.square(tmp_test_data[:,14]*data_std[14]+data_mean[14] - mac_spd))))

#     prediction2 = np.zeros((t_max, 8))
#     input_data_raw = tmp_test_data[[0], :]
#     print(input_data_raw)
#     for t in range(1, t_max):
# #         input_data[0,3] = 0
#         input_data = (input_data_raw - data_mean) / data_std
#         input_data_torch = torch.from_numpy(input_data).float()
#         output_data_torch = model_cpu(input_data_torch)
#         prediction2[t] = output_data_torch.data.numpy() * label_std + label_mean + input_data_raw[:, [0, 1, 12, 13, 14, 15, 16, 17]]
#         input_data_raw = np.concatenate([prediction2[t, [0, 1]], tmp_test_data[t, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]], prediction2[t, [2, 3, 4, 5, 6, 7]]])

        

#         input_data_raw[1] = np.mod(input_data_raw[1]+np.pi, 2*np.pi) - np.pi
#         input_data_raw = input_data_raw.reshape(1, -1)
#     print(prediction2.shape)

#     for q in quantities:
#             plt.figure(figsize=(12, 8))
#             plt.plot(tmp_test_data[:t_max,q], 'r', label='Truth')
#             plt.plot(np.arange(1,t_max), prediction2[1:, quantities_transform[q]], 'b', label='Pred')
#             plt.plot([0, t_max], [tmp_test_data[0,q], tmp_test_data[0,q]], 'g-.', label='Benchmark')
#             plt.legend()
#             plt.grid(True)
#         #     plt.ylim([0.97, 1.03])
#             plt.title('#{:d}: [{:s}] Pred MSE={:.4e} Benchmark MSE={:.4e}'.format(i, quantites_names[q], np.mean(np.square(prediction2[1:, quantities_transform[q]] - tmp_test_data[1:t_max,q].reshape(-1))), np.mean(np.square(tmp_test_data[0,q] - tmp_test_data[1:t_max,q].reshape(-1))) ))
#             print(tmp_test_data[0,q])
    
# plt.show()